In [ ]:
import numpy as np
import sys
import os

from astropy import units as u
from astropy.time import Time

import theano.tensor as tt
import theano
import pymc3 as pm

import exoplanet as xo

import starry

sys.path.append("../volcano")
from utils import get_body_ephemeris

np.random.seed(42)

starry.config.lazy = True

In [ ]:
%matplotlib inline
%run notebook_setup.py

## PCA model for the time variable map

Given the data consisting of $l=1\dots L$ lightcurves with $T$ data points each and assuming that each light curve was generated from a different map specified with a spherical coefficient vector $\mathbf{y}_l$, the predicted flux for the $l$-th lightcurve is given by

$$
\mathbf{f}_l=\mathbf{A}_l\,\mathbf{y}_l
$$

where $\mathbf{f}_l$ of shape $(T\times 1)$ is the observed flux, $\mathbf{A}_l$  is the design matrix $(T\times N)$ and $\mathbf{y}_l$ is an $(N\times 1) $ vector of SH coefficients.
We can write the predicted flux for all lightcurves simultaneously as

$$
\mathbf{f}=\mathbf{A}\mathbf{y}
$$
where $\mathbf{f}$ is $(TL\times 1)$, $\mathbf{A}$ is a block diagonal matrix of the form

$$
\mathbf{A}=\left(\begin{array}{cccccc}
\mathbf{A}_1 & & & & & \\
& \mathbf{A}_2 & & & \\
& & \mathbf{A}_3 & & \\
& & &  \ddots\\
& & & &  \mathbf{A}_L \\
\end{array}\right)
$$
with shape $(TL\times NL)$ and $\mathbf{y}$ is $(NL\times 1)$.


Insted of fitting for all $N*L$  coefficients simultaneously, we can reduce the number of parameters by treating the map generating any given light curve as a linear combination of $K$ basis maps.
We first reshape the vector $\mathbf{y}$ into a matrix $\mathbf{Y}$ of shape $(N\times L)$. 
We then decompose this matrix as a product of two matrices $\mathbf{P}$ and $\mathbf{Q}$ where the columns of $\mathbf{P}$ define the basis maps and columns of $\mathbf{Q}$ specify the linear coefficients for each lightcruve $l$.

$$
\mathbf{Y}=\mathbf{P}\,\mathbf{Q}
$$

where $P$ is $(N\times K)$ and $Q$ is $(K\times L)$. 
The total number of parameters in our model is then $NK+KL$. 
To compute the predicted flux $\mathbf{f}$, we unroll the matrix $\mathbf{Y}$ into a vector of the same shape
as $\mathbf{y}$, that is, $\mathbf{y}'=\mathrm{vec}\,(\mathbf{Y})$ where $\mathrm{vec}$ denotes the unroll operation by stacking the columns of the $\mathbf{Y}$ matrix.
This unrolled vector  has the following form

$$
\mathbf{y}'=\left(\begin{array}{c}
\sum_kq_{k1}\mathbf{p}_k \\
\sum_kq_{k2}\mathbf{p}_k \\
\vdots \\
\sum_kq_{kL}\mathbf{p}_k
\end{array}\right)
$$

where $\mathbf{p}_k$ is the $k$-th column of the matrix $\mathbf{P}$. 




Although the problem is not linear in both $\mathbf{P}$ and $\mathbf{Q}$ simulataneously, it is linear in each matrix separately. 
Assuming that $\mathbf{P}$ is known, we have 

$$
\mathbf{f}=\mathbf{A}\,\mathbf{P}'\,\mathbf{q}
$$

where $\mathbf{q}\equiv\mathrm{vec}(\mathbf{Q})$ and $\mathbf{P}'$ is an $(NL\times KL)$ matrix constructed by repeating the $\mathbf{P}$ matrix  $L$ times on the diagonal:

$$
\mathbf{P}'=\left(\begin{array}{cccccc}
\mathbf{P} & & & & & \\
& \mathbf{P} & & & \\
& & \mathbf{P} & & \\
& & &  \ddots\\
& & & &  \mathbf{P} \\
\end{array}\right)
$$

Given a prior mean $\boldsymbol{\mu}_{\mathbf{q}}$ and covariance $\boldsymbol{\Sigma}_{\mathbf{q}}$ the analytic solution for the MAP estimate of the posterior is

$$
\widehat{\mathbf{q}}=\boldsymbol{\Sigma}_{\widehat{\mathbf{q}}}\left(\mathbf{P}'^{\top} \mathbf{A}^{\top} \boldsymbol{\Sigma}_{\mathbf{f}}^{-1} \mathbf{f}+\mathbf{\Lambda}_{\mathbf{q}}^{-1} \boldsymbol{\mu}_{\mathbf{q}}\right)
$$

where

$$
\boldsymbol{\Sigma}_{\widehat{\mathbf{q}}}=\left(\mathbf{P}'^{\top} \mathbf{A}^{\top} \boldsymbol{\Sigma}_{\mathbf{f}}^{-1} \mathbf{A} \mathbf{P}'+\mathbf{\Lambda}_{\mathbf{q}}^{-1}\right)^{-1}
$$

If instead we know $\mathbf{Q}$, we have

$$
\mathbf{f}=\mathbf{A}\,\mathbf{Q}'\,\mathbf{p}
$$

where $\mathbf{p}\equiv\mathrm{vec}(\mathbf{P})$ and $\mathbf{Q}'$ is an $(NL\times NK)$ matrix constructed as

$$
\mathbf{Q}'=\left(\begin{array}{c}
q_{11} \mathbf{I}_{N} & q_{21}\mathbf{I}_{N} & \ldots & q_{K1} \mathbf{I}_{N}\\
q_{12} \mathbf{I}_{N} & q_{22}\mathbf{I}_{N} & \ldots & q_{K2} \mathbf{I}_{N}\\
\vdots & \vdots & \ddots & \vdots\\
q_{1L} \mathbf{I}_{N} & q_{2L}\mathbf{I}_{N} & \ldots & q_{KL} \mathbf{I}_{N}\\
\end{array}\right)
$$

where $\mathbf{I}_{N}$ is the identity matrix of size $N$. Given a prior mean $\boldsymbol{\mu}_{\mathbf{p}}$ and covariance $\boldsymbol{\Sigma}_{\mathbf{p}}$ the posterior is analytic as for $\mathbf{q}$.

In [ ]:
# Load the data
times = np.load("times.npy")
A_np = np.load("A_matrix.npy")
Y_true = np.load("Y_matrix.npy")
true_P_matrix = np.load("P_matrix.npy")
flux_true = np.load("flux_model.npy")
t_spot = np.load("t_spot.npy")
spot1_amp = np.load("spot1_amp.npy")
spot2_amp = np.load("spot2_amp.npy")

flux_err = 0.01 * np.ones(len(flux_true))
flux_obs = flux_true + np.random.normal(0, 0.005, size=(len(flux_true)))

In [ ]:
# Plot flux
fig, ax = plt.subplots(figsize=(15, 4))
ax.plot(flux_obs, "C0.")

## Fit for the Q matrix assuming known basis matrix

In [ ]:
map = starry.Map(ydeg=8)

with pm.Model() as model:

    A = pm.Data("A", A_np)

    N = map.Ny

    # Decompose L maps with N coefficients each into a product of two A_list,
    # Y = PQ where P is (N, K) and Q is (K, L)
    K = 4  # nr. of basis maps
    L = len(Y_true[0, :])  # nr of light curves

    # P matrix fixed to the one used for generating the data
    P = pm.Data("P", true_P_matrix)

    # Columns of Q are time dependent coeffcients
    mu_Q = np.zeros(L)
    cov_Q = 5 * np.identity(L)
    Q = pm.MvNormal(
        "Q", mu_Q, cov_Q, shape=(K, L), testval=tt.as_tensor_variable(np.zeros((K, L)))
    )

    # Compute the full matrix of coefficients for each lightcurve
    y = tt.dot(P, Q)
    pm.Deterministic("y", y)

    # Unroll
    y = tt.flatten(y.T)

    # Predicted flux
    flux_model = pm.Deterministic("flux_model", tt.dot(A, y))

    pm.Normal("obs", mu=flux_model, sd=flux_err, observed=flux_obs)

In [ ]:
%%time
with model:
    map_soln = xo.optimize()

In [ ]:
res = flux_obs - map_soln["flux_model"]
res_rms = np.sqrt(np.mean(res ** 2))
print("Residuals RMS: ", res_rms)

fig, ax = plt.subplots(2, 1, figsize=(15, 6), sharex=True)

ax[0].plot(flux_obs[::5], "ko", alpha=0.2)
ax[0].plot(map_soln["flux_model"][::5], "C1-", alpha=0.8)
ax[0].set_title("MAP solution, known P")

ax[1].plot(res[::5], "ko", alpha=0.2)

for a in ax.flatten():
    a.grid()
    a.set_xlim(0, 5000)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

im = ax.imshow(map_soln["Q"])
cbar = fig.colorbar(im)
ax.set_title("MAP estimate for Q")

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(6, 6), sharex=True)
ax[0].plot(t_spot - t_spot[0], map_soln["Q"][0, :], "C0o", label="row1")
ax[0].plot(t_spot - t_spot[0], map_soln["Q"][1, :], "C1o", label="row2")
# ax[0].plot(t_spot - t_spot[0], map_soln['Q'][2, :], 'C2o', label='row3')
# ax[0].plot(t_spot - t_spot[0], map_soln['Q'][3, :], 'C3o', label='row4')

ax[0].legend()
ax[0].set_title("MAP estimate of the Q matrix")

ax[1].plot(t_spot - t_spot[0], spot1_amp, "C0o", label="spot1")
ax[1].plot(t_spot - t_spot[0], spot2_amp, "C1o", label="spot2")
ax[1].set_xlabel("time [days]")
ax[1].legend()
ax[1].set_title("Amplitudes of the two spots [truth]")

for a in ax.flatten():
    a.grid()

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
ax.imshow(map_soln["y"])
ax.set_title("MAP Y matrix")

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

idx = 20
map.reset()
map.amp = tt.as_tensor_variable(map_soln["y"][0, idx])
map[1:, :] = tt.as_tensor_variable(map_soln["y"][1:, idx])
img = map.render(projection="rect")
extent = (-180, 180, -90, 90)
ax.set_yticks([-90, -60, -30, 0, 30, 60, 90])
ax.set_xticks([-180, -90, 0, 90, 180])
ax.imshow(img.eval(), origin="lower", extent=extent, cmap="plasma")
ax.set_title(f"MAP estimate for light curve {idx + 1}")
ax.set_xlabel("Longitude [deg]")
ax.set_ylabel("Latitutde [deg]")

## Fitting for both P and Q

In [ ]:
map = starry.Map(ydeg=8)

with pm.Model() as model:

    A = pm.Data("A", A_np)

    N = map.Ny

    # Decompose L maps with N coefficients each into a product of two A_list,
    # Y = PQ where P is (N, K) and Q is (K, L)
    K = 3  # nr. of basis maps
    L = len(Y_true[0, :])  # nr of light curves

    # Columns of P are the K basis maps, we ignore the first row of the matrix
    # because this will be modeled seperately
    mu_P = np.zeros(N - 1)
    cov_P = 1e-1 * np.identity(N - 1)
    P_ = pm.MvNormal(
        "P_",
        mu_P,
        cov_P,
        shape=(K, N - 1),
        testval=tt.as_tensor_variable(np.zeros((K, N - 1))),
    ).T

    # The first row of the P matrix defines the amplitudes of the basis maps
    ln_amp = pm.MvNormal(
        "ln_amp",
        np.zeros(K),
        10 * np.identity(K),
        shape=(K,),
        testval=np.random.rand(K),
    )
    P = tt.concatenate([tt.exp(ln_amp[None, :]), P_], axis=0)
    pm.Deterministic("P", P)

    # Q matrix
    mu_Q = np.zeros(L)
    cov_Q = 2 * np.identity(L)
    Q = pm.MvNormal(
        "Q",
        mu_Q,
        cov_Q,
        shape=(K, L),
        testval=tt.as_tensor_variable(np.random.rand(K, L)),
    )

    # Compute the full matrix of coefficients for each lightcurve
    y = tt.dot(P, Q)
    pm.Deterministic("y", y)

    # Unroll
    y = tt.flatten(y.T)

    # Predicted flux
    flux_model = tt.dot(A, y)

    pm.Deterministic("flux_model", flux_model)
    pm.Normal("obs", mu=flux_model, sd=flux_err, observed=flux_obs)

In [ ]:
# Initialize at the true values with a small perturbation
init = {
    "ln_amp": np.log(true_P_matrix[0, :3]) + 1e-02 * np.log(true_P_matrix[0, :3]),
    "P_": true_P_matrix[1:, :3].T + 1e-04 * true_P_matrix[1:, :3].T,
    "Q": map_soln["Q"][:3, :],
}

In [ ]:
%%time
with model:
    start = xo.optimize(start=init, vars=[model.ln_amp, model.P_])
    start = xo.optimize(start=start, vars=[model.Q])
    start = xo.optimize(start=start, vars=[model.ln_amp, model.P_])
    start = xo.optimize(start=start, vars=[model.Q])
    map_soln2 = xo.optimize(start=start)

In [ ]:
res = flux_obs - map_soln2["flux_model"]
res_rms = np.sqrt(np.mean(res ** 2))
print("Residuals RMS: ", res_rms)

fig, ax = plt.subplots(2, 1, figsize=(15, 6), sharex=True)

ax[0].plot(flux_obs[::5], "ko", alpha=0.2)
ax[0].plot(map_soln2["flux_model"][::5], "C1-", alpha=0.8)
ax[0].set_title("MAP solution, known P")

ax[1].plot(res[::5], "ko", alpha=0.2)

for a in ax.flatten():
    a.grid()
    a.set_xlim(0, 5000)

In [ ]:
%%time
with model:
    start = xo.optimize(vars=[model.ln_amp])
    map_soln2 = xo.optimize(start=start)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))

ax.plot(flux_obs, "ko", alpha=0.5)
ax.plot(map_soln2["flux_model"], "C1-", alpha=0.8)
ax.set_xlim(0, 10000)
ax.grid()
ax.set_title("Maximum likelihood solution")

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

im = ax.imshow(map_soln2["P"])
cbar = fig.colorbar(im)
ax.set_title("MAP estimate for P")

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

im = ax.imshow(map_soln2["Q"])
cbar = fig.colorbar(im)
ax.set_title("MAP estimate for Q")

In [ ]:
spot1_amp = np.load("spot1_amp.npy")
spot2_amp = np.load("spot2_amp.npy")


fig, ax = plt.subplots(2, 1, figsize=(5, 6), sharex=True)
ax[0].plot(t_spot - t_spot[0], map_soln2["Q"][0, :], "C0o", label="row1")
ax[0].plot(t_spot - t_spot[0], map_soln2["Q"][1, :], "C1o", label="row2")
# ax[0].plot(t_spot - t_spot[0], map_soln['Q'][2, :], 'C2o', label='row3')
# ax[0].plot(t_spot - t_spot[0], map_soln['Q'][3, :], 'C3o', label='row4')

ax[0].legend()
ax[0].set_title("MAP estimate of the Q matrix")

ax[1].plot(t_spot - t_spot[0], spot1_amp, "C0o", label="spot1")
ax[1].plot(t_spot - t_spot[0], spot2_amp, "C1o", label="spot2")
ax[1].set_xlabel("time [days]")
ax[1].legend()
ax[1].set_title("Amplitudes of the two spots [truth]")

for a in ax.flatten():
    a.grid()

In [ ]:
# Plot basis maps
fig, ax = plt.subplots(3, 1, figsize=(6, 12))

for i in range(3):
    map = starry.Map(ydeg=8)
    map.amp = tt.as_tensor_variable(map_soln2["P"][0, i])
    map[1:, :] = tt.as_tensor_variable(map_soln2["P"][1:, i])
    img = map.render(projection="rect")
    extent = (-180, 180, -90, 90)
    ax[i].set_yticks([-90, -60, -30, 0, 30, 60, 90])
    ax[i].set_xticks([-180, -90, 0, 90, 180])
    ax[i].imshow(img.eval(), origin="lower", extent=extent, cmap="plasma")
    ax[i].set_title(f"Component {i + 1}")

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

idx = 15
map.reset()
map.amp = tt.as_tensor_variable(map_soln2["y"][0, idx])
map[1:, :] = tt.as_tensor_variable(map_soln2["y"][1:, idx])
img = map.render(projection="rect")
extent = (-180, 180, -90, 90)
ax.set_yticks([-90, -60, -30, 0, 30, 60, 90])
ax.set_xticks([-180, -90, 0, 90, 180])
ax.imshow(img.eval(), origin="lower", extent=extent, cmap="plasma")
ax.set_title(f"MAP estimate for light curve {idx + 1}")
ax.set_xlabel("Longitude [deg]")
ax.set_ylabel("Latitutde [deg]")